# Generate Full Disk images of GOES-R NetCDF files and upload them to GitLab database.

In [ ]:
import sys
from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

